In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools 
import json 
from scipy.ndimage import gaussian_filter
import sqlite3
import ast
import matplotlib.patches as patches

mariter = itertools.cycle(('o','s','^','p'))
# marker = next(mariter) # paste this to cycle through differt marker styles if needed

plt.rcParams['figure.figsize'] = [9,6] # size of the plot; use 3t:2t ratio; recommended 9:6 or 15:10
plt.rcParams['figure.dpi'] = 500 # quality of the plot; use 300 for debugging, 1000+ for quality
plt.rcParams['axes.titlesize'] = 22 # fontsize of the title
plt.rcParams['axes.labelsize'] = 18 # fontsize of the x- and y-axis labels
plt.rcParams['xtick.labelsize'] = 14 # fontsize of the x-axis
plt.rcParams['ytick.labelsize'] = 14 # fontsize of the y-axis
plt.rcParams['legend.markerscale'] = 0.5 # legend marker size
plt.rcParams['lines.markersize'] = 10 # marker size
plt.rcParams['lines.linestyle'] = '--' # line in between points; use 'None' or '--' (dashed)
plt.rcParams['lines.linewidth'] = 1 # width of line between points, if any
plt.rcParams['axes.titlepad'] = 15 # distance between plot area and title
plt.rcParams['axes.labelpad'] = 10 # distance between plot area and title
plt.rcParams['font.family'] = 'Helvetica' # font of plot text
plt.rcParams['axes.linewidth'] = 2 # thickness of plot border
plt.rcParams['xtick.major.size'] = 10  # length of major ticks on x-axis
plt.rcParams['xtick.minor.size'] = 3  # length of minor ticks on x-axis
plt.rcParams['ytick.major.size'] = 10  # length of major ticks on y-axis
plt.rcParams['ytick.minor.size'] = 3  # length of minor ticks on y-axis

In [ ]:
# SQLITE3 FILES

file_path = '/Users/amrtayamen/Downloads/AT_EUM_007_01_A_dZ-60_dX-70_step_0p5_halfcy-2_waves-1000.sqlite3'

# experiment parameters

step_size = 0.5

num_row = int(60/step_size) # dZ for X-Z scan, dY for X-Y scan
num_col = int(70/step_size) # dX for X-Z scan, dX for X-Y scan

# Connect to the SQLite database file
conn = sqlite3.connect(file_path)

# Create a cursor object
cursor = conn.cursor()

# Query the database to select 'X' and 'Z' columns from a specific table
cursor.execute("SELECT X, Z, voltage, time FROM acoustics")

# Fetch all the rows
rows = cursor.fetchall()

# Close the connection
conn.close()

# Convert the fetched rows to a 2D NumPy array
data_array = np.array(rows)

time = np.array(ast.literal_eval(data_array[0,3]), dtype=float)/1e3
voltage = np.array(ast.literal_eval(data_array[251,2]), dtype=float)

wave_mesh = data_array[:,2].reshape(num_row+1,num_col+1)

ref_row = 0
ref_col = 0

ref_waveform = np.array(ast.literal_eval(wave_mesh[ref_row,ref_col]), dtype=float)

dot_prod = np.zeros((num_row+1,num_col+1))

ref_prod = np.zeros((num_row+1,num_col+1))

P = 895.2022009

for i in range(0,num_row+1):
    for j in range(0,num_col+1):
        waveform = np.array(ast.literal_eval(wave_mesh[i,j]), dtype=float)
        # dot_prod[i,j] = np.dot(waveform, waveform)
        # ref_prod[i,j] = np.dot(waveform, ref_waveform)
        dot_prod[i,j] = waveform.max() - waveform.min()
        # ref_prod[i,j] = 1/(20*(1-math.sqrt(1-(dot_prod[i,j]/P)*math.exp(-77.27857408*0.015))))
        # dot_prod[i,j] = waveform.max()

In [ ]:
plt.imshow(dot_prod[:,:], cmap=plt.cm.viridis_r, interpolation='nearest',vmin=q, vmax=q+1) # vmin = 1000, vmax = 4500
plt.xticks(ticks=list(range(0, num_col+1, 10)), labels=list(range(0,  1*int(num_col*step_size)+1, 5)))
plt.yticks(ticks=list(range(0, num_row+1, 10)), labels=list(range(0,  1*int(num_row*step_size)+1, 5)))
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')
cbar = plt.colorbar()
# plt.title('Acoustic image from min-max difference')
plt.show()
